In [17]:
import glob
import os
import shutil

In [19]:
path_img=glob.glob("data_sets/proc/moviescope/img/*")

In [20]:
len(path_img)

3449

In [8]:
path_new = glob.glob("data_sets/proc/moviescope/train_wrong/*")

In [14]:
for p in path_new:
    p=p.replace("train_wrong","img").replace(".","_a.")
    os.remove(p)

In [18]:
for p in path_new:
    pn=p.replace("train_wrong","img").replace(".","_a.")
    shutil. copy(p, pn)

In [5]:
from models.gpt2image import GPT2ClfImage
from train import *

In [6]:
from PIL import ImageFile, Image
from torchvision.transforms import functional as TF

In [7]:
import torch.nn as nn

In [8]:
import argparse
args = argparse.Namespace(
        bert_model="bert-base-uncased",
        data_path="data_sets/proc/",
        task="moviescope",
        model="gpt2image",
        batch_sz=8,
        task_type="multilabel",
        aug=0,
        drop_img_percent=0,
        max_seq_len=512,
        num_image_embeds=3,
        n_workers=8,
        poster="raw",
        dropout=0.1,
        hidden_sz=768,
        n_classes=13,
        uda=1,
        seed=1,
        savedir="checkpoints2",
        name ="gpt2_1",
        glove_path="glove.6B.300d.txt",
        freeze_img=0,
        freeze_txt=0,
        
        
    )

In [9]:
model=GPT2ClfImage(args)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Restored from ../data_augmentation/imgnt.ckpt


In [10]:
model.cuda()

GPT2ClfImage(
  (tok_emb): Embedding(16384, 1536)
  (drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=1536, out_features=1536, bias=True)
        (query): Linear(in_features=1536, out_features=1536, bias=True)
        (value): Linear(in_features=1536, out_features=1536, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (resid_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1536, out_features=1536, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=1536, out_features=6144, bias=True)
        (1): GELU()
        (2): Linear(in_features=6144, out_features=1536, bias=True)
        (3): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((1536,), eps=1e-05, elemen

In [26]:
model.to(device) 

GPT2ClfImage(
  (tok_emb): Embedding(16384, 1536)
  (drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=1536, out_features=1536, bias=True)
        (query): Linear(in_features=1536, out_features=1536, bias=True)
        (value): Linear(in_features=1536, out_features=1536, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (resid_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1536, out_features=1536, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=1536, out_features=6144, bias=True)
        (1): GELU()
        (2): Linear(in_features=6144, out_features=1536, bias=True)
        (3): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((1536,), eps=1e-05, elemen

In [24]:
model=model.cuda()

In [11]:
train_loader, val_loader, test_loader,train_loader_unsup = get_data_loaders(args)

In [9]:
bs=[]
for batch in tqdm(train_loader, total=len(train_loader)):
    bs.append(batch)

100%|██████████| 432/432 [00:03<00:00, 123.39it/s]


In [19]:
img_tensor=bs[0][3]

In [8]:
criterion= nn.BCEWithLogitsLoss()
for batch in tqdm(train_loader, total=len(train_loader)):
    loss, _, _ = model_forward(1, model, args, criterion, batch)

  0%|          | 0/432 [00:00<?, ?it/s]


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [12]:
batch=[]
for b in train_loader:
    batch.append(b)

In [16]:
batch[0][3].shape

torch.Size([8, 3, 256, 256])

In [17]:
img_tensor = batch[0][3]

In [18]:
model(img_tensor)

RuntimeError: Input, output and indices must be on the current device

In [15]:
img_tensor.shape

torch.Size([8, 3, 256, 256])

In [4]:
train(args)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Restored from ../data_augmentation/imgnt.ckpt


AttributeError: 'Namespace' object has no attribute 'weight_classes'

In [13]:
pil_image = Image.open("../Multimodal-Transformers/data_sets/proc/moviescope/MatchedPosters/0.jpg").convert('RGB')
pil_image = pil_image.resize((256, 256))
image=(TF.to_tensor(pil_image)* 2 - 1).unsqueeze(0)

In [29]:
img=b[3].cuda()
model(img)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [23]:
img_tensor.shape

torch.Size([8, 3, 256, 256])

In [15]:
_, z_indices = model.encode(image)
token_embeddings=model.tok_emb(z_indices)
t = token_embeddings.shape[1]
position_embeddings =model.pos_emb[:, :t, :] # each position maps to a (learnable) vector
x = model.drop(token_embeddings + position_embeddings)
x = model.blocks(x)
x = model.ln_f(x)


In [24]:
out = model.classifier(x[:,-1,:])

In [26]:
out.shape

torch.Size([1, 13])

In [56]:
import glob
import os
import json
import jsonlines
from transformers import BertTokenizer
import nltk.data

In [6]:
path_img=glob.glob("data_sets/proc/moviescope/sinteticas2/*")

In [40]:
path_img

['data_sets/proc/moviescope/sinteticas2/3157_Romance.jpg',
 'data_sets/proc/moviescope/sinteticas2/2741_Action.jpg',
 'data_sets/proc/moviescope/sinteticas2/366_Action.jpg',
 'data_sets/proc/moviescope/sinteticas2/4198_Romance.jpg',
 'data_sets/proc/moviescope/sinteticas2/4569_Family.jpg',
 'data_sets/proc/moviescope/sinteticas2/1442_Romance.jpg',
 'data_sets/proc/moviescope/sinteticas2/3177_Family.jpg',
 'data_sets/proc/moviescope/sinteticas2/1468_Biography.jpg',
 'data_sets/proc/moviescope/sinteticas2/2219_Drama.jpg',
 'data_sets/proc/moviescope/sinteticas2/4740_Crime.jpg',
 'data_sets/proc/moviescope/sinteticas2/5021_Comedy.jpg',
 'data_sets/proc/moviescope/sinteticas2/1764_Romance.jpg',
 'data_sets/proc/moviescope/sinteticas2/2545_Drama.jpg',
 'data_sets/proc/moviescope/sinteticas2/3081_Comedy.jpg',
 'data_sets/proc/moviescope/sinteticas2/1860_Horror.jpg',
 'data_sets/proc/moviescope/sinteticas2/2745_Thriller.jpg',
 'data_sets/proc/moviescope/sinteticas2/4110_Family.jpg',
 'data_se

In [9]:
ids=[]
for p in path_img:
    ids.append(os.path.split(p)[-1])

In [19]:
for i in range(len(ids)):
    ids[i]=ids[i].replace("_"," ").split(" ")[0]

In [26]:
ids=list(set(ids))

In [28]:
data=[json.loads(l) for l in open("data_sets/proc/moviescope/train.jsonl")]

In [32]:
data_filtrados=[]
for d in data:
    if str(d['id']) in ids:
        data_filtrados.append(d)

In [38]:
with jsonlines.open("data_sets/proc/moviescope/train_s.jsonl",'w') as writer:
    writer.write_all(data_filtrados)

In [39]:
data_filtrados

[{'id': 100,
  'label': ['Action', 'Crime', 'Thriller'],
  'title': 'The Fast and the Furious\xa0',
  'plot_keywords': 'eighteen wheeler|illegal street racing|truck|trucker|undercover cop',
  'synopsis': 'In the docks outside Los Angeles, a semi-truck is loaded with electronics. On the road, it is attacked by three heavily modified black Honda Civics with green underglow. The occupants rob the electronics and escape. The next day, undercover LAPD officer Brian O\'Conner is assigned to find the gang responsible for the crimes. O\'Conner uses his cover job at a chop shop to infiltrate the L.A. street racing scene. While visiting Torreto\'s Market, a local grocery, he flirts with the shop\'s owner, Mia Toretto, who is the younger sister of a well-known street racer, Dominic Toretto. Vince, one of Dominic’s henchmen and who has a crush on her, starts a fight with O’Conner, which Dominic breaks up. During a local racing gathering, O’Conner arrives with his 1995 Mitsubishi Eclipse GS, and Do

In [73]:
data_aumentados=[]
for d in data_filtrados:
    data_aumentados.append(d)
    sentences=tokenizer.tokenize(d['synopsis'])
    sz=0
    if len(d["label"])>0:
        sz=int(len(sentences)/len(d["label"]))
    i=0
    for g in d["label"]:
        text=" ".join(sentences[sz*i:sz*(i+1)])
        i+=1
        aug={"id":str(d["id"])+"_"+g,"label":[g],'title':d['title'],'plot_keywords':d['plot_keywords'],'synopsis':text}
        data_aumentados.append(aug)

In [74]:
len(data_aumentados)

4328

In [75]:
with jsonlines.open("data_sets/proc/moviescope/train_aug_s.jsonl",'w') as writer:
    writer.write_all(data_aumentados)

In [51]:
tok=BertTokenizer.from_pretrained("bert-base-uncased")

In [57]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [60]:
sentences=tokenizer.tokenize(data_aumentados[0]['synopsis'])

In [1]:
from models.bert import ClfDalle
from models.bert import BertClfImg 
from models.bert import BertClfDalle
from models.bert import LongBertClf
from models.bert import ImgBertClf

import argparse

from utils.helpers import get_data_loaders


In [1]:
import torch.nn as nn
import torch
import os 
# HuggingFace
from transformers import BertTokenizer, BertModel, BertConfig, BertForMaskedLM

In [2]:
bert = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text="HuggingFace is based in NYC [SEP] Where is HuggingFace based?"

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
inputs = tokenizer(text, return_tensors="pt", padding='max_length',truncation=True)

In [31]:
outputs = bert(**inputs)

In [33]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2183, -0.2746, -0.0711,  ..., -0.7341,  0.2405,  0.6967],
         [ 0.2269, -0.8810,  1.2781,  ...,  0.4240,  0.2847,  0.0934],
         [ 0.3613, -0.1709,  0.4845,  ..., -0.2132, -0.5260, -0.1214],
         ...,
         [-0.4029, -0.3080, -0.2850,  ...,  0.2659,  0.4385, -0.2281],
         [-0.1528, -0.2970,  0.0968,  ...,  0.0738,  0.2685, -0.2301],
         [-0.2154, -0.2600,  0.1809,  ...,  0.1650,  0.3116, -0.1064]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-0.8925, -0.4194, -0.7642,  0.8162,  0.5629, -0.2678,  0.8750,  0.3602,
         -0.6660, -1.0000, -0.3375,  0.8509,  0.9631,  0.4367,  0.8955, -0.8214,
         -0.3824, -0.5806,  0.3579, -0.5491,  0.5926,  0.9999,  0.2097,  0.3853,
          0.4961,  0.9509, -0.8216,  0.9007,  0.9403,  0.7606, -0.7267,  0.2211,
         -0.9743, -0.2318, -0.8168, -0.9811,  0.3996, -0.7480,  0.0956, -0.0328,
         -0.8152,  0.3683,  0.999

In [23]:
_, x = bert(
            input_ids=inputs.input_ids,
            token_type_ids=inputs.token_type_ids,
            attention_mask=inputs.attention_mask,
            return_dict=False,
        )

In [25]:
x.shape

torch.Size([1, 768])

In [4]:
from utils.utils import *

In [2]:
args = argparse.Namespace(
        bert_model="bert-base-uncased",
        data_path="data_sets/proc/",
        task="moviescope",
        model="clf_dalle",
        batch_sz=8,
        task_type="multilabel",
        aug=0,
        drop_img_percent=0,
        max_seq_len=512,
        num_image_embeds=3,
        n_workers=8,
        poster="raw",
        dropout=0.1,
        hidden_sz=768,
        n_classes=13,
        uda=1
        
    )

In [3]:
model=ClfDalle(args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
load_checkpoint2(model, os.path.join("checkpoints/emb_dalle_1", "model_best.pt"))

In [14]:
W=model.embeddings.weight

In [22]:
import torch.nn as nn

In [19]:
torch.save(W,"checkpoints/emb_dalle_1/w.pt")

In [20]:
torch.load("checkpoints/emb_dalle_1/w.pt")

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.9317,  0.0737,  0.2112,  ...,  1.8661, -0.0834, -1.5330],
        [ 0.7093,  0.6281,  1.7030,  ...,  1.0653, -0.3637,  0.0312],
        ...,
        [ 0.3812, -0.6700,  0.0836,  ...,  0.0991,  0.4768,  0.7695],
        [-0.2999,  0.2834,  0.0229,  ..., -0.4517,  2.3942,  1.2709],
        [ 0.1412,  0.3649, -0.8488,  ..., -0.0069, -1.1461,  0.2279]],
       requires_grad=True)

In [24]:
E=nn.Embedding.from_pretrained(torch.load("checkpoints/emb_dalle_1/w.pt"),freeze=False)

In [26]:
E.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.9317,  0.0737,  0.2112,  ...,  1.8661, -0.0834, -1.5330],
        [ 0.7093,  0.6281,  1.7030,  ...,  1.0653, -0.3637,  0.0312],
        ...,
        [ 0.3812, -0.6700,  0.0836,  ...,  0.0991,  0.4768,  0.7695],
        [-0.2999,  0.2834,  0.0229,  ..., -0.4517,  2.3942,  1.2709],
        [ 0.1412,  0.3649, -0.8488,  ..., -0.0069, -1.1461,  0.2279]],
       requires_grad=True)

In [45]:
model.bert=model.bert.requires_grad_(False)

In [46]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
            {"params": [p for n, p in param_optimizer if p.requires_grad and not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
            {"params": [p for n, p in param_optimizer if p.requires_grad and any(nd in n for nd in no_decay)], "weight_decay": 0.0,},
        ]

In [47]:
optimizer_grouped_parameters

[{'params': [Parameter containing:
   tensor([[ 0.0088,  0.0204, -0.0023,  ...,  0.0339, -0.0135, -0.0344],
           [ 0.0237,  0.0262,  0.0094,  ..., -0.0117, -0.0149,  0.0259],
           [ 0.0117,  0.0218,  0.0078,  ...,  0.0244,  0.0117, -0.0022],
           ...,
           [-0.0162,  0.0066, -0.0236,  ..., -0.0026, -0.0299,  0.0031],
           [ 0.0277,  0.0339, -0.0174,  ...,  0.0293,  0.0154,  0.0019],
           [ 0.0255, -0.0306,  0.0042,  ...,  0.0355,  0.0330, -0.0135]],
          requires_grad=True)],
  'weight_decay': 0.01},
 {'params': [Parameter containing:
   tensor([-0.0199,  0.0316,  0.0267, -0.0119,  0.0062,  0.0333,  0.0059,  0.0031,
            0.0195, -0.0262, -0.0219, -0.0102, -0.0035], requires_grad=True)],
  'weight_decay': 0.0}]

In [33]:
parameters=filter(lambda p: p.requires_grad, model.parameters())

In [35]:
param_optimizer = [p for p in model.parameters() if p.requires_grad]

In [36]:
param_optimizer

[Parameter containing:
 tensor([[-0.0299,  0.0292,  0.0038,  ...,  0.0352,  0.0090,  0.0274],
         [-0.0019, -0.0233, -0.0165,  ...,  0.0322,  0.0130,  0.0190],
         [-0.0112, -0.0349, -0.0149,  ..., -0.0021, -0.0323, -0.0038],
         ...,
         [-0.0189, -0.0261,  0.0158,  ...,  0.0011,  0.0075, -0.0071],
         [ 0.0223,  0.0351,  0.0266,  ..., -0.0278,  0.0278, -0.0116],
         [ 0.0355,  0.0145,  0.0013,  ...,  0.0188, -0.0277,  0.0040]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0197,  0.0051, -0.0277, -0.0095,  0.0212,  0.0089, -0.0322, -0.0359,
          0.0038,  0.0104, -0.0036, -0.0355,  0.0088], requires_grad=True)]

In [26]:
param_optimizer = list(model.named_parameters())

In [31]:
param_optimizer[0]

('bert.embeddings.word_embeddings.weight',
 Parameter containing:
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0038, -0.0144,  0.0172,  ..., -0.0034, -0.0071,  0.0309],
         [-0.0060, -0.0533, -0.0016,  ..., -0.0033, -0.0040,  0.0137],
         ...,
         [ 0.0112,  0.0012, -0.0181,  ...,  0.0077,  0.0227,  0.0134],
         [ 0.0284, -0.0170, -0.0272,  ..., -0.0100, -0.0170,  0.0127],
         [ 0.0069,  0.0019, -0.0101,  ..., -0.0108, -0.0195, -0.0089]]))

In [9]:
import torchvision.transforms as transforms
from PIL import Image, ImageFile
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True
import glob
import torch
from PIL import Image, ImageStat
import json

In [12]:
data = [json.loads(l) for l in open("data_sets/proc/moviescope/train.jsonl")]
for d in data:
    poster = Image.open("data_sets/proc/moviescope/MatchedPosters/"+str(d["id"])+".jpg").convert("RGB")
    poster = poster.resize((256,256))
    poster = transforms.ToTensor()(poster)
    list_img.append(poster)

In [13]:
mi=torch.cat(list_img,dim=2)
mi = mi.view(mi.size(0), -1)

In [5]:
mi.mean(1)

tensor([0.4713, 0.4191, 0.3896])

In [14]:
mi.mean(1)

tensor([0.4691, 0.4171, 0.3883])

In [6]:
mi.std(1)

tensor([0.3563, 0.3416, 0.3374])

In [15]:
mi.std(1)

tensor([0.3564, 0.3415, 0.3371])